In [96]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
toro= pd.read_csv('/Users/nikhilshah/desktop/projects/toro_df.csv')
toro.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West , India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [4]:
CLIENT_ID = 'AF3CDTZRGQZ02CPUBEYU105LZLGEZOOQ0FM3005PUAUBM10T' # your Foursquare ID
CLIENT_SECRET = 'MURNYC2HE5DRANFJUB5QQA4HU21Y3N4RVXAEMVKDTIQOT4J4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [68]:
def getnearbyvenues(names,latitude,longitude,radius=500, limit=100):
    
    place_list=[]
    for name, lat, long in zip(names,latitude, longitude):
        
    
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        limit
        )
    
        results= requests.get(url).json()['response']['groups'][0]['items']
    
        place_list.append([(
            name,
            lat,
            long,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],    
            v['venue']['categories'][0]['name']) for v in results])
    
    places_df= pd.DataFrame([item for place in place_list for item in place])
    places_df.columns=['Postcode',
            'Borough latitude',
            'Borough longitude',
            'Place',
            'Place Latitude',
            'Place Longitude',
            'Place Category']

    
    return(places_df)
    

In [69]:
toro_places= getnearbyvenues(names=toro['Postcode'],latitude=toro['Latitude'],longitude=toro['Longitude'])
toro_places.head()

,Postcode,Borough latitude,Borough longitude,Place,Place Latitude,Place Longitude,Place Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [75]:
toro_places.groupby('Postcode').count()

,Borough latitude,Borough longitude,Place,Place Latitude,Place Longitude,Place Category
Postcode,,,,,,
M4E,4,4,4,4,4,4
M4K,42,42,42,42,42,42
M4L,18,18,18,18,18,18
M4M,38,38,38,38,38,38
M4N,3,3,3,3,3,3
M4P,9,9,9,9,9,9
M4R,16,16,16,16,16,16
M4S,38,38,38,38,38,38
M4T,2,2,2,2,2,2


In [76]:
len(toro_places['Place Category'].unique())

237

In [81]:
toro_places_onehot= pd.get_dummies(toro_places[['Place Category']], prefix='', prefix_sep='')
toro_places_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
toro_places_onehot['Postcode']=toro_places['Postcode']
fix_col= [toro_places_onehot.columns[-1]]+list(toro_places_onehot.columns[:-1])
toro_places_onehot=toro_places_onehot[fix_col]
toro_places_onehot.head()

,Postcode,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
toro_grouped = toro_places_onehot.groupby('Postcode').mean().reset_index()
toro_grouped

,Postcode,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,...,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.023810
2,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.026316
4,M4N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.062500
7,M4S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,...,0.00000,0.00,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.00,0.000000


In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [105]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toro_grouped['Postcode']

for ind in np.arange(toro_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Pub,Trail,Neighborhood,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Brewery,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,M4L,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,Seafood Restaurant
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [106]:
k=3

toro_kmean= toro_grouped.drop('Postcode',1)

k_mean=KMeans(n_clusters=k, random_state=5).fit(toro_kmean)

k_mean.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 0, 0, 1, 0], dtype=int32)

In [107]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', k_mean.labels_)

toro_merged = toro

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toro_merged = toro_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

toro_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Neighborhood,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Brewery,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,M4L,East Toronto,"The Beaches West , India Bazaar",43.668999,-79.315572,0,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [93]:
toro_merged.shape

(38, 16)

In [108]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toro_merged['Latitude'], toro_merged['Longitude'], toro_merged['Postcode'], toro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [109]:
toro_merged.loc[toro_merged['Cluster Labels'] == 0, toro_merged.columns[[0] + list(range(5, toro_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,0,Health Food Store,Pub,Trail,Neighborhood,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4K,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Brewery,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,M4L,0,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,Seafood Restaurant
5,M4P,0,Playground,Hotel,Clothing Store,Food & Drink Shop,Park,Sandwich Place,Grocery Store,Gym,Breakfast Spot,Event Space
6,M4R,0,Coffee Shop,Yoga Studio,Bagel Shop,Park,Clothing Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Diner,Salon / Barbershop
7,M4S,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Thai Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Deli / Bodega
9,M4V,0,Coffee Shop,Pub,Liquor Store,Sushi Restaurant,Supermarket,Sports Bar,Fried Chicken Joint,Light Rail Station,American Restaurant,Vietnamese Restaurant
11,M4X,0,Coffee Shop,Restaurant,Pizza Place,Café,Bakery,Pub,Park,Italian Restaurant,Japanese Restaurant,Breakfast Spot
12,M4Y,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Gay Bar,Men's Store,Gym,Pub,Mediterranean Restaurant,Bubble Tea Shop


In [110]:
toro_merged.loc[toro_merged['Cluster Labels'] == 1, toro_merged.columns[[0] + list(range(5, toro_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,1,Playground,Tennis Court,Concert Hall,Convenience Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [111]:
toro_merged.loc[toro_merged['Cluster Labels'] == 2, toro_merged.columns[[0] + list(range(5, toro_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,2,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
10,M4W,2,Park,Playground,Trail,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
23,M5P,2,Sushi Restaurant,Park,Jewelry Store,Trail,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
